In [ ]:
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import re
import string
import itertools

In [ ]:
# initialize parameter settings
parameter_names = ['eta','no_self_supervised','balance_theory']
parameter_names_display = [r'$\eta$','no self supervised', 'balance theory'] # for matplotlib
parameter_values = [[0, 0.05,0.1,0.15,0.2, 0.25],
                    [True, False],
                    [True, False]]
parameter_values_display = [[0, 0.05,0.1,0.15,0.2, 0.25],
                    ['no '+r'$\mathcal{L}_{PBNC}$','w/ '+r'$\mathcal{L}_{PBNC}$'],
                    ['social balance theory', 'SSSNET']]
parameter_dict = dict(zip(parameter_names, parameter_values))
parameter_dict_display = dict(zip(parameter_names, parameter_values_display))
compare_names = ['A','L','L_sym','BNC','BRC','SPONGE','SPONGE_sym','SSSNET']
figure_markers = ['*','P','<','s','8','D','>','v']
size_ratio = 1.5
link_sign_loss_ratio = 0.1
K = 5
p = 0.02
total_n = 1050
K = 2
N = 200
num_com = 2
p = 0.1
lr = 0.01
tau = 0.5
hop = 2
seed_ratio = 0.1
triplet_loss_ratio = 0.1
hidden = 32
alpha = 0
supervised_loss_ratio = 50
link_sign_loss = False
feat_choice = 'A_reg'
method_str = 'SpectralSSSNET'

In [ ]:
def hyper_polarized_semi_comparison_plot(change_parameter = 'eta', default_choice_indices = [0,1,1], balance_num=3, save=False):
    '''
    ablation study and hyperparameter analysis
    '''
    change_var_ind = parameter_names.index(change_parameter) # to be shown in x-variable in the plots
    change_var_values = parameter_dict[change_parameter]
    var_name_display = parameter_names_display[change_var_ind]
    default_values = [parameter_values[i][j] for i, j in enumerate(default_choice_indices)] # initialize default values
    # extract results
    results_mean = np.zeros([1,len(compare_names),len(change_var_values)])
    results_std = np.zeros([1,len(compare_names),len(change_var_values)])
    cut_results_mean = np.zeros([len(compare_names),len(change_var_values),balance_num])
    cut_results_std = np.zeros([len(compare_names),len(change_var_values),balance_num])
    for i, var in enumerate(change_var_values):
        updated_values = default_values.copy()
        updated_values[change_var_ind] = var
        eta,no_self_supervised, balance_theory = updated_values # update parameter settings
        param_values = [total_n, num_com, p,eta,K,N,hop,tau,size_ratio, seed_ratio, alpha, lr, 
                        hidden, triplet_loss_ratio, link_sign_loss, link_sign_loss_ratio, supervised_loss_ratio]
        file_name = '../result_arrays/polarized/test'+'_'.join([str(int(100*value)) for value in param_values])+\
        '_'+feat_choice+'_'+method_str
        if no_self_supervised:
            file_name += 'no_self_supervised'
        if balance_theory:
            file_name += 'balance_theory'
        file_name += '.npy'
        res = None
        try:
            res = np.load(file_name)
        except FileNotFoundError:
            print(file_name)
            results_mean[:,:,i] = np.nan
            results_std[:,:,i] = np.nan
        if res is not None:
            for j in range(1):
                res_piece = res[:,j*(len(compare_names)):(j+1)*(len(compare_names))]
                results_mean[j, -1,i] = np.nanmean(res_piece[:,-1], axis=0)
                results_std[j, -1,i] = np.nanstd(res_piece[:,-1], axis=0)
        # balanced_cuts
        file_name = '../result_arrays/polarized/balanced_cuts'+'_'.join([str(int(100*value)) for value in param_values])+\
        '_'+feat_choice+'_'+method_str
        if no_self_supervised:
            file_name += 'no_self_supervised'
        if balance_theory:
            file_name += 'balance_theory'
        file_name += '.npy'
        res = None
        try:
            res = np.load(file_name)
            res[:,:,2] = 100 * res[:,:,2]
            cut_results_mean[:,i] = np.nanmean(res, axis=0)
            cut_results_std[:,i] = np.nanstd(res, axis=0)
            all_not_found = False
        except FileNotFoundError:
            # print(file_name+' not found')
            cut_results_mean[:,i] = np.nan
            cut_results_std[:,i] = np.nan
        except ValueError:
            print(file_name+' value error')
            cut_results_mean[:,i] = np.nan
            cut_results_std[:,i] = np.nan
            
    return(results_mean[0,-1,:], results_std[0,-1,:], cut_results_mean[-1,:], cut_results_std[-1,:])

In [ ]:
def plot_hyper(change_parameter = 'no_self_supervised', default_choice_indices = [0,1,1], save=True):
    balance_num = 3
    change_parameter_base = 'eta'
    change_var_ind_base = parameter_names.index(change_parameter_base) # to be shown in x-variable in the plots
    change_var_values_base = parameter_dict[change_parameter_base]
    var_name_display_base = parameter_names_display[change_var_ind_base]
    default_values = [parameter_values[i][j] for i, j in enumerate(default_choice_indices)] # initialize default values
    change_var_ind = parameter_names.index(change_parameter) # to be shown in x-variable in the plots
    change_var_values = parameter_dict[change_parameter]
    change_var_values_display = parameter_dict_display[change_parameter]
    var_name_display = parameter_names_display[change_var_ind]
    # extract results
    results_mean = np.zeros([len(change_var_values),len(change_var_values_base)])
    results_std = np.zeros([len(change_var_values),len(change_var_values_base)])
    cut_results_mean = np.zeros([len(change_var_values),len(change_var_values_base),balance_num])
    cut_results_std = np.zeros([len(change_var_values),len(change_var_values_base),balance_num])
    plt.figure(figsize=[5,4])
    plt.rcParams.update({'font.size': 16})
    for i in range(len(change_var_values)):
        updated_choice_indices = default_choice_indices.copy()
        updated_choice_indices[change_var_ind] = i
        results_mean[i], results_std[i], cut_results_mean[i], cut_results_std[i] = hyper_polarized_semi_comparison_plot(change_parameter = 'eta', 
                                                                                                                        default_choice_indices = updated_choice_indices, 
                                                                                                                        balance_num=balance_num)
        plt.errorbar(change_var_values_base, results_mean[i], yerr=results_std[i], label=change_var_values_display[i],alpha=0.8, fmt=figure_markers[i], ls='None')
    plt.legend(loc='best',framealpha=0.1,fontsize=14)
    plt.xlabel(var_name_display_base, labelpad=-5)
    plt.ylabel('ARI', labelpad=-5)
    plt.rcParams.update({'font.size': 16})    
    title_name = var_name_display
    save_name_base = '_'.join([str(int(100*value)) for value in default_values])+'Change_{}'.format(change_parameter)
    dir_name = '../comparison_plots/ablation/'
    save_name = dir_name+'test'+save_name_base
    save_name += '.pdf'
    if save:
        plt.savefig(save_name)
    plt.title(title_name)
    plt.show()
    print(results_mean)
    
    
    y_labels = [r'$\mathcal{L}_{BRC}$', r'$\mathcal{L}_{BNC}$', 'unhappy ratio(%)']
    save_y = ['BRC', 'BNC', 'unhappy_ratio']
    change_var_values_base = np.array(change_var_values_base)
    for i in range(balance_num):  
        plt.figure(figsize=[5,4])
        plt.xticks(np.arange(change_var_values_base.min(),change_var_values_base.max()+0.1,step=0.1))
        plt.rcParams.update({'font.size': 17})
        for j in range(len(change_var_values)):
            plt.errorbar(change_var_values_base, cut_results_mean[j,:,i], yerr=cut_results_std[j,:,i], label=change_var_values_display[j],alpha=0.8, fmt=figure_markers[i], ls='None')
        plt.legend(loc='best',framealpha=0.1,fontsize=18)
        plt.xlabel(var_name_display_base, labelpad=-5)
        plt.ylabel(y_labels[i], labelpad=-5)
        if save_y[i] == 'BNC':
            plt.ylim(2.1,2.3)

        if save:
            print('Saving figure!')
            save_name = dir_name + save_y[i]+save_name_base
            save_name += '.pdf'
            plt.savefig(save_name,format='pdf')
        plt.title(title_name)
        plt.show()
        print(cut_results_mean[:,:,i])

In [ ]:
plot_hyper('no_self_supervised')

In [ ]:
def plot_hyper(change_parameter = 'no_self_supervised', default_choice_indices = [0,1,1], save=True):
    balance_num = 2
    change_parameter_base = 'eta'
    change_var_ind_base = parameter_names.index(change_parameter_base) # to be shown in x-variable in the plots
    change_var_values_base = parameter_dict[change_parameter_base]
    var_name_display_base = parameter_names_display[change_var_ind_base]
    default_values = [parameter_values[i][j] for i, j in enumerate(default_choice_indices)] # initialize default values
    change_var_ind = parameter_names.index(change_parameter) # to be shown in x-variable in the plots
    change_var_values = parameter_dict[change_parameter]
    change_var_values_display = parameter_dict_display[change_parameter]
    var_name_display = parameter_names_display[change_var_ind]
    # extract results
    results_mean = np.zeros([len(change_var_values),len(change_var_values_base)])
    results_std = np.zeros([len(change_var_values),len(change_var_values_base)])
    cut_results_mean = np.zeros([len(change_var_values),len(change_var_values_base),balance_num])
    cut_results_std = np.zeros([len(change_var_values),len(change_var_values_base),balance_num])
    plt.figure(figsize=[5,4])
    plt.rcParams.update({'font.size': 17})
    for i in range(len(change_var_values)):
        updated_choice_indices = default_choice_indices.copy()
        updated_choice_indices[change_var_ind] = i
        results_mean[i], results_std[i], cut_results_mean[i], cut_results_std[i] = hyper_polarized_semi_comparison_plot(change_parameter = 'eta', 
                                                                                                                        default_choice_indices = updated_choice_indices, 
                                                                                                                        balance_num=balance_num)
        plt.errorbar(change_var_values_base, results_mean[i], yerr=results_std[i], label=change_var_values_display[i],alpha=0.8, fmt=figure_markers[i], ls='None')
    plt.legend(loc='best',framealpha=0.1,fontsize=18)
    plt.xlabel(var_name_display_base, labelpad=-5)
    plt.ylabel('ARI', labelpad=-5)
    plt.rcParams.update({'font.size': 17})    
    title_name = var_name_display
    save_name_base = '_'.join([str(int(100*value)) for value in default_values])+'Change_{}'.format(change_parameter)
    dir_name = '../comparison_plots/ablation/'
    save_name = dir_name+'test'+save_name_base
    save_name += '.pdf'
    if save:
        plt.savefig(save_name)
    plt.title(title_name)
    plt.show()
    print(results_mean)
    
    
    y_labels = [r'$\mathcal{L}_{BRC}$', r'$\mathcal{L}_{BNC}$', 'unhappy ratio']
    save_y = ['BRC', 'BNC', 'unhappy_ratio']
    change_var_values_base = np.array(change_var_values_base)
    for i in range(balance_num):  
        plt.figure(figsize=[5,4])
        plt.xticks(np.arange(change_var_values_base.min(),change_var_values_base.max()+0.1,step=0.1))
        plt.rcParams.update({'font.size': 17})
        for j in range(len(change_var_values)):
            plt.errorbar(change_var_values_base, cut_results_mean[j,:,i], yerr=cut_results_std[j,:,i], label=change_var_values_display[j],alpha=0.8, fmt=figure_markers[i], ls='None')
        plt.legend(loc='best',framealpha=0.1,fontsize=18)
        plt.xlabel(var_name_display_base, labelpad=-5)
        plt.ylabel(y_labels[i],labelpad=-5)
        if save_y[i] == 'BNC':
            plt.ylim(2.1,2.3)

        if save:
            print('Saving figure!')
            save_name = dir_name + save_y[i]+save_name_base
            save_name += '.pdf'
            plt.savefig(save_name,format='pdf')
        plt.title(title_name)
        plt.show()
        print(cut_results_mean[:,:,i])

In [ ]:
plot_hyper('balance_theory')